In [1]:
import netCDF4
import numpy as np
from datetime import datetime, timedelta

# Define the time variables
start_date = datetime(2019, 7, 1)
num_days = 31
dye_time = np.arange(0, num_days * 86400, 900)
dye_time_units = 'seconds since ' + start_date.strftime('%Y-%m-%d %H:%M:%S')

# Load the ROMS grid information
grid = netCDF4.Dataset('TXLA_grd.nc')
lat_rho = grid.variables['lat_rho'][:]
lon_rho = grid.variables['lon_rho'][:]

# Define the dye concentration values
dye_conc = np.zeros((len(dye_time), grid.dimensions['s_rho'].size,
                     grid.dimensions['eta_rho'].size, grid.dimensions['xi_rho'].size))

# Calculate the indices of the grid cell nearest to the specified location
lat_idx = np.argmin(np.abs(lat_rho[:, 0] - 28.852141))
lon_idx = np.argmin(np.abs(lon_rho[0, :] - -92.814206))

# Set the dye concentration values for the specified location
dye_conc[:, :, lat_idx, lon_idx] = 1.0

# Create the forcing file and write the variables
forcing_file = netCDF4.Dataset('dye_forcing.nc', 'w', format='NETCDF4')

forcing_file.createDimension('xi_rho', grid.dimensions['xi_rho'].size)
forcing_file.createDimension('eta_rho', grid.dimensions['eta_rho'].size)
forcing_file.createDimension('s_rho', grid.dimensions['s_rho'].size)
forcing_file.createDimension('time', None)

lon_rho_var = forcing_file.createVariable('lon_rho', 'f8', ('eta_rho', 'xi_rho'))
lat_rho_var = forcing_file.createVariable('lat_rho', 'f8', ('eta_rho', 'xi_rho'))
dye_time_var = forcing_file.createVariable('dye_time', 'f8', ('time',))
dye_conc_var = forcing_file.createVariable('dye_conc', 'f8',
                                           ('time', 's_rho', 'eta_rho', 'xi_rho'),
                                           fill_value=-9999.0)

lon_rho_var[:] = lon_rho
lat_rho_var[:] = lat_rho
dye_time_var[:] = dye_time
dye_time_var.units = dye_time_units
dye_conc_var[:] = dye_conc

forcing_file.close()


FileNotFoundError: [Errno 2] No such file or directory: b'TXLA_grd.nc'